In [8]:
import yfinance as yf
import pandas as pd
from tsq.data_card import save_data_card_html
import IPython.display as display

In [2]:
# 1. Download raw OHLCV data
df_raw = yf.download("AAPL", start="2020-01-01", end="2020-12-31")
df_raw.head()

C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\2871068594.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_raw = yf.download("AAPL", start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.538528,72.598907,71.292319,71.545905,135480400
2020-01-03,71.833290,72.594055,71.608685,71.765667,146322800
2020-01-06,72.405670,72.444313,70.703005,70.954181,118387200
2020-01-07,72.065155,72.671348,71.845377,72.415345,108872000
2020-01-08,73.224396,73.526287,71.768071,71.768071,132079200


In [3]:

def to_tidy_timeseries(df_raw: pd.DataFrame, entity_name: str | None = None) -> pd.DataFrame:
    """
    Convert OHLCV-like data to tidy long format:
    columns -> ['timestamp', 'entity', 'variable', 'value'].

    Handles:
      - MultiIndex columns (level0=field, level1=ticker)  e.g., ('Close','AAPL')
      - Single-level columns (field only), with entity_name provided
    Index is assumed to be a DatetimeIndex (or convertible).
    """
    df = df_raw.copy()

    # Ensure timestamp column name
    if df.index.name is None:
        df.index.name = "timestamp"
    else:
        # normalize common index names to 'timestamp'
        df.index.name = "timestamp"

    # Convert index to a column
    df = df.reset_index()

    if isinstance(df_raw.columns, pd.MultiIndex):
        # After reset_index, columns are mixed: 'timestamp' + MultiIndex columns for data
        # Stack by the ticker level to get ticker as a column
        # detect which level is the ticker (usually level 1)
        # assume level 0 = field, level 1 = ticker (as in your example)
        data_cols = df.columns[df.columns != "timestamp"]
        # Build a DataFrame of only the multiindex columns, keep timestamp aside
        df_data = df.drop(columns=["timestamp"])

        # Stack by level=1 (ticker) -> index becomes [..., ticker], columns become fields
        df_long = df_data.stack(level=1).reset_index()
        df_long.columns = ["level_0", "entity"] + list(df_long.columns[2:])  # rename the stacked name to 'entity'
        # 'level_0' is the row index from df_data before stack; map it back to timestamps
        # But easier: re-merge with the timestamp from the original reset df using position
        # Instead, rebuild using the original df with set_index on timestamp then stack:
        df2 = df_raw.copy()
        if df2.index.name is None:
            df2.index.name = "timestamp"
        df2 = df2.stack(level=1)  # (timestamp, ticker) index; columns=fields
        df2.index = df2.index.set_names(["timestamp", "entity"])
        df2 = df2.reset_index()   # columns: ['timestamp','entity', fields...]

        # Melt fields into variable/value
        tidy = df2.melt(id_vars=["timestamp", "entity"], var_name="variable", value_name="value")
        # Ensure datetime
        tidy["timestamp"] = pd.to_datetime(tidy["timestamp"], errors="coerce")
        return tidy[["timestamp", "entity", "variable", "value"]]

    else:
        # Single-level columns (e.g., ['Open','High','Low','Close','Adj Close','Volume'])
        if entity_name is None:
            raise ValueError(
                "Single-level columns detected. Please provide `entity_name` (e.g., ticker)."
            )
        # Melt all non-timestamp columns
        tidy = df.melt(id_vars=["timestamp"], var_name="variable", value_name="value")
        tidy["entity"] = entity_name
        tidy["timestamp"] = pd.to_datetime(tidy["timestamp"], errors="coerce")
        return tidy[["timestamp", "entity", "variable", "value"]]



df_tidy = to_tidy_timeseries(df_raw)

print(df_tidy.head())

   timestamp entity variable      value
0 2020-01-02   AAPL    Close  72.538528
1 2020-01-03   AAPL    Close  71.833290
2 2020-01-06   AAPL    Close  72.405670
3 2020-01-07   AAPL    Close  72.065155
4 2020-01-08   AAPL    Close  73.224396


C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:30: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_data = df.drop(columns=["timestamp"])
C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:33: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_long = df_data.stack(level=1).reset_index()
C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df2 = df2.stack(level=1)  # (timestamp, ticker) index; columns=fields


In [4]:
save_data_card_html(df_tidy, "Apple OHLC", "apple_data_card.html")


In [9]:
# show saved html file in notebook

display.HTML("apple_data_card.html")


Variable,Obs,Missing,Min,Max
Close,252,0,54.31694412231445,133.190185546875
High,252,0,55.316762499666766,135.23642465149393
Low,252,0,51.47000835368413,130.90036509315357
Open,252,0,55.21508661799568,134.51538107187807
Volume,252,0,46691300.0,426510000.0
Variable,Obs,Missing,Min,Max
Close,252,0,54.31694412231445,133.190185546875
High,252,0,55.316762499666766,135.23642465149393
Low,252,0,51.47000835368413,130.90036509315357
Open,252,0,55.21508661799568,134.51538107187807


In [5]:
# tickers
tickers = ["AAPL", "MSFT", "GOOGL"]


df_raw_multi = yf.download(tickers, start="2020-01-01", end="2020-12-31")
df_raw_multi.head()

C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\3793201825.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_raw_multi = yf.download(tickers, start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  3 of 3 completed


Price           Close                              High             \
Ticker           AAPL      GOOGL        MSFT       AAPL      GOOGL   
Date                                                                 
2020-01-02  72.538528  68.026024  152.791107  72.598907  68.026024   
2020-01-03  71.833290  67.670158  150.888611  72.594055  68.278016   
2020-01-06  72.405670  69.473839  151.278625  72.444313  69.499184   
2020-01-07  72.065155  69.339645  149.899307  72.671348  69.756647   
2020-01-08  73.224396  69.833183  152.287018  73.526287  70.171653   

Price                         Low                              Open  \
Ticker            MSFT       AAPL      GOOGL        MSFT       AAPL   
Date                                                                  
2020-01-02  152.895746  71.292319  66.923141  150.612732  71.545905   
2020-01-03  152.153786  71.608685  66.964392  150.355908  71.765667   
2020-01-06  151.345221  70.703005  67.147292  148.881450  70.954181   
2020-01-07  151.887434  71.845377  69.163206  149.651985  72.415345   
2020-01-08  152.962418  71.768071  69.216384  150.251325  71.768071   

Price                                 Volume                      
Ticker          GOOGL        MSFT       AAPL     GOOGL      MSFT  
Date                                                              
2020-01-02  67.018569  151.040795  135480400  27278000  22622100  
2020-01-03  66.998193  150.603245  146322800  23408000  21116200  
2020-01-06  67.178599  149.423674  118387200  46768000  20813700  
2020-01-07  69.605552  151.554502  108872000  34330000  21634100  
2020-01-08  69.325228  151.183554  132079200  35314000  27746500

In [6]:
df_tidy_multi = to_tidy_timeseries(df_raw_multi)

print(df_tidy_multi.head())

   timestamp entity variable       value
0 2020-01-02   AAPL    Close   72.538528
1 2020-01-02  GOOGL    Close   68.026024
2 2020-01-02   MSFT    Close  152.791107
3 2020-01-03   AAPL    Close   71.833290
4 2020-01-03  GOOGL    Close   67.670158


C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:30: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_data = df.drop(columns=["timestamp"])
C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:33: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_long = df_data.stack(level=1).reset_index()
C:\Users\JanderM\AppData\Local\Temp\ipykernel_41864\971069261.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df2 = df2.stack(level=1)  # (timestamp, ticker) index; columns=fields


In [7]:
save_data_card_html(df_tidy_multi, f"{tickers} OHLC", "multi_data_card.html")

In [10]:
# show saved html file in notebook

display.HTML("multi_data_card.html")

Variable,Obs,Missing,Min,Max
Close,756,0,52.39228439331055,222.11190795898438
High,756,0,53.02747485160064,223.2720931821289
Low,756,0,50.1427750747484,217.9889702596058
Open,756,0,52.503616205473385,219.82989336183493
Volume,756,0,9312000.0,426510000.0
Variable,Obs,Missing,Min,Max
Close,252,0,54.31694412231445,133.190185546875
High,252,0,55.316762499666766,135.23642465149393
Low,252,0,51.47000835368413,130.90036509315357
Open,252,0,55.21508661799568,134.51538107187807
